In [2]:
import logging

import rich
import openai

import guardrails as gd

In [3]:
guard = gd.Guard.from_rail('guardrails/prompt_v2.xml')

/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter 1-indexed is not valid for element integer.
  warnings.warn(
/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter percentage is not valid for element float.
  warnings.warn(
/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter tone-twitter explain-high-quality is not valid for element string.
  warnings.warn(
/Users/shreyarajpal/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter tagline tv-ad is not valid for element string.
  warnings.warn(


In [4]:
guard.output_schema.fees.children.item.children.name.validators[0].on_fail

<bound method Validator.noop of <guardrails.validators.LowerCase object at 0x7ff4b0730a90>>

In [5]:
rich.print(guard.base_prompt)

Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 
'None'.

{document}


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="name" format="lower-case; two-words"/>
            <string name="explanation" format="one-line"/>
            <float name="value" format="percentage"/>
            <string name="description" format="length: 0 200"/>
            <string name="example" required="True" format="tone-twitter explain-high-quality"/>
            <string name="advertisement" format="tagline tv-ad"/>
        </object>
    </list>
    <string name="interest_rates" description="What are the interest rates offered by the bank on savings and 
checking accounts, loans, and credit products?" format="one-line"/>
</output>




Return a valid JSON object that respects this XML format and extracts only the information requested in this 
document. Respect the types indicated in the XML -- the information you extract should be converted into the 
correct 'type'. Try to be as correct and concise as possible. Find all relevant information in the document. If you
are unsure of the answer, enter 'None'. If you answer incorrectly, you will be asked again until you get it right 
which is expensive.

In [6]:
content = gd.docs_utils.read_pdf('data/chase_card_agreement.pdf')
# response, response_as_dict, validated_response = guard.ask_with_validation(content[:6000])

In [7]:
print(guard.base_prompt)



Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{document}


Given below is XML that describes the information to extract from this document and the tags to extract it into.


<output>
    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="name" format="lower-case; two-words"/>
            <string name="explanation" format="one-line"/>
            <float name="value" format="percentage"/>
            <string name="description" format="length: 0 200"/>
            <string name="example" required="True" format="tone-twitter explain-high-quality"/>
            <string name="advertisement" format="tagline tv-ad"/>
        </object>
    </list>
    <string name="interest_rates" description="What are the interest rates offered by the bank on savings and checking accounts, loans, and cr

In [9]:
response, validated_response = guard(openai.Completion.create, prompt_params={'document': content[:6000]}, engine='text-davinci-003', max_tokens=2048, temperature=0)

In [10]:
rich.print(validated_response)

{
    'fees': [
        {
            'index': 1,
            'name': 'annual membership',
            'explanation': 'annual membership fee',
            'value': 0,
            'description': 'None',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 2,
            'name': 'maximum recursion',
            'explanation': 'fixed finance charge',
            'value': 1.72,
            'description': 'Monthly fee of 0% of the amount of each eligible purchase transaction or amount 
selected to create a My Chase Plan while in the 0% Intro Purchase APR period. After that, monthly fee of 1.72% of 
the amount of each eligible purchase transaction or amount selected to create a My Chase Plan. The My Chase Plan 
Fee will be determined at the time each My Chase Plan is created and will remain the same until the My Chase Plan 
is paid in full.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 3,
            'name': 'balance transfers',
            'explanation': 'intro fee',
            'value': 5,
            'description': 'Intro fee of either $5 or 3% of the amount of each transfer, whichever is greater, on 
transfers made within 60 days of account opening. After that: Either $5 or 5% of the amount of each transfer, 
whichever is greater.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 4,
            'name': 'cash advances',
            'explanation': 'transaction fee',
            'value': 5,
            'description': 'Either $10 or 5% of the amount of each transaction, whichever is greater.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 5,
            'name': 'foreign transactions',
            'explanation': 'transaction fee',
            'value': 3,
            'description': '3% of the amount of each transaction in U.S. dollars.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 6,
            'name': 'late payment',
            'explanation': 'penalty fee',
            'value': 40,
            'description': 'Up to $40.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 7,
            'name': 'depth exceeded',
            'explanation': 'penalty fee',
            'value': 0,
            'description': 'None',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 8,
            'name': 'return payment',
            'explanation': 'penalty fee',
            'value': 40,
            'description': 'Up to $40.',
            'example': 'None',
            'advertisement': 'None'
        },
        {
            'index': 9,
            'name': 'return check',
            'explanation': 'penalty fee',
            'value': 0,
            'description': 'None',
            'example': 'None',
            'advertisement': 'None'
        }
    ],
    'interest_rates': 'Purchase Annual Percentage Rate (APR) 0% Intro APR for the first 18 months that your Account
is open. After that, 19.49%. This APR will vary with the market based on the Prime Rate. My Chase Loan SM APR 
19.49%. This APR will vary with the market based on the Prime Rate. Balance Transfer APR 0% Intro APR for the first
18 months that your Account is open. After that, 19.49%. This APR will vary with the market based on the Prime 
Rate. Cash Advance APR 29.49%. This APR will vary with the market based on the Prime Rate. Penalty APR and When It 
Applies Up to 29.99%. This APR will vary with the market based on the Prime Rate.'
}

## How to add versioning

In [18]:
rail_str = """
<aiml version="0.1">
<prompt>

Given the following document, answer the following questions. If the answer doesn't exist in the document, enter 'None'.

{{document}}

@xml_prefix_prompt

{output_schema}

@json_suffix_prompt</prompt>


<output>
    <list name="fees" description="What fees and charges are associated with my account?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="lower-case; two-words" on-fail-lower-case="noop" on-fail-two-words="reask"/>
            <string name="explanation" format="one-line" on-fail-one-line="noop" />
            <float name="value" format="percentage"/>
            <string name="description" format="length: 0 200" on-fail-length="noop" />
            <string name="example" required="True" format="tone-twitter explain-high-quality" />
            <string name="advertisement" format="tagline tv-ad" />
        </object>
    </list>
    <string name='interest_rates' description='What are the interest rates offered by the bank on savings and checking accounts, loans, and credit products?' format="one-line" on-fail-one-line="noop"/>
</output>

</aiml>
"""

In [14]:
from lxml import etree as ET

In [19]:
parser = ET.XMLParser(encoding="utf-8")
parsed_rail = ET.fromstring(rail_str, parser=parser)

In [20]:
parsed_rail.attrib

{'version': '0.1'}